In [41]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.1333, -0.0328,  0.1088,  0.0659,  0.1813, -0.0404, -0.2509, -0.0495,
          0.0720,  0.1604],
        [ 0.0517,  0.1889,  0.1232,  0.0550,  0.0359,  0.0324, -0.2779, -0.0812,
          0.0635,  0.2494]], grad_fn=<AddmmBackward0>)

In [42]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, x):
        return self.out(F.relu(self.hidden(x)))
    
    
net = MLP()
net(X)

tensor([[-0.0962,  0.1512,  0.0717,  0.3104,  0.2300, -0.2528,  0.0292, -0.1037,
         -0.2938, -0.2282],
        [ 0.0385,  0.1148, -0.0149,  0.1609,  0.2715, -0.3421,  0.2248, -0.0029,
         -0.3013, -0.2040]], grad_fn=<AddmmBackward0>)

In [43]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, model in enumerate(args):
            self._modules[str(idx)] = model
            
    def forward(self,x):
        for block in self._modules.values():
            x = block(x)
        return x
    
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0269,  0.0106,  0.0500, -0.1761,  0.1180,  0.2983,  0.0184, -0.0080,
         -0.1633,  0.1765],
        [ 0.1351,  0.1751,  0.0080, -0.1717,  0.0660,  0.4083,  0.0329, -0.0402,
         -0.2071,  0.1684]], grad_fn=<AddmmBackward0>)

In [44]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
    
    
net = FixedHiddenMLP()
net(X)

tensor(0.0830, grad_fn=<SumBackward0>)

In [45]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0175, grad_fn=<SumBackward0>)